# Beamline 2-1 Samples

In [10]:
import sys, os
sys.path.append(os.path.join('..','src'))

from helper_functions import * 
from helper_functions_topas import * 
 
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

### Load in data from XY files 

Here X is stored as a dictionary of dataframes corresponding to each XY file. Y comes from a csv which contain the true lattice parameters, crystal system and wavelength

In [8]:
X = np.load("../data/Xdata.npy",allow_pickle=True).item()
Y = pd.read_csv("../data/ssrl_labels.csv")

In [9]:
Y.head()

,Material,Crystal System,a,b,c,alpha,beta,gamma,Wavelength
0,BL21Robot_0001,cubic,4.156826,4.156826,4.156826,90,90.0,90,0.729568
1,BL21Robot_0002,trigonal,4.914155,4.914155,5.405698,90,90.0,120,0.729568
2,BL21Robot_0096,orthorhombic,7.612981,7.787221,10.654601,90,90.0,90,0.729568
3,BL21Robot_0105,hexagonal,3.248316,3.248316,5.204099,90,90.0,120,0.729568
4,BL21Robot_0106,cubic,10.114644,10.114644,10.114644,90,90.0,90,0.729568


Example: Generate the Topas input file for the first structure with bound = 0.2. The text, starting at continue_after_convergence, can be copied and pasted into Topas and run. The running directory needs to contain the corresponding file from /data/XYdata. 

In [5]:
bound = 0.2
row = Y.iloc[0,:]
name = row['Material']
wavelength = row['Wavelength']
crystal_system = row['Crystal System']
Xdata = X[name]
y_true = np.sort(row[['a','b','c']])
    
prediction = np.ravel(predictExptDataPipeline(Xdata, y_true, crystal_system=crystal_system, measured_wavelength=wavelength, showPlots=False,printResults=False))
print("Analysing sample: {}, with crystal system: {} and wavelength: {}".format(name, crystal_system, wavelength))
print("Predicted LPs: ", prediction)
print("True LPs: ", np.array(row[['a','b','c','alpha','beta','gamma']]))


Analysing sample: BL21Robot_0001, with crystal system: cubic and wavelength: 0.7295684
Predicted LPs:  [4.0949674 4.0949674 4.0949674]
True LPs:  [4.156826 4.156826 4.156826 90 90.0 90]


In [6]:
# Makes the template file for each structure
make_template(prediction=prediction, name=name,wavelength=wavelength,crystal_system=crystal_system,bound=bound)


    continue_after_convergence
    verbose 0

    XY(BL21Robot_0001,0.01)
        bkg @ 0 0 0 0 0 0 0
        start_X 15
        finish_X 30
        Zero_Error(@, 0.00074)
        lam
            ymin_on_ymax  0.0001
            la  1 lo 0.7295684 lh 0.002

        LP_Factor(90)
        x_calculation_step 0.01
        gauss_fwhm  @  0.0337972443 min 0.01 max 0.5
        lor_fwhm @  0.084548646` min 0.01 max 0.5

    hkl_Is

        lp_search 1
        scale 0.01
        prm end = If(Get(r_wp) < 0.1, 0, 5000);
        iters = end;

        volume 68.66751098632812 min 35.15776800256879 max 118.65746700866967
        a @ 4.094967365264893 min 3.275973892211914 max 4.9139608383178714
        b = Get(a);
        c = Get(a);

        space_group 195
    